# Registration

In [1]:
import xarray as xr
import numpy as np

import cedalion
import cedalion.io
import cedalion.dataclasses as cdc
import cedalion.geometry.registration
import cedalion.geometry.segmentation
import cedalion.plots

xr.set_options(display_expand_data=False);

## Read optode locations from snirf file

Optode locations are returned as a 2D xr.DataArray. Different labeled points are found along the first dimension 'label'. The second dimensions contains the 3D coordinates of each point. There is an abundance of coordinate system (CRS) definitions and in this example alone we have to distinguish between these different coordinate system:
- the segmented volume is in voxel space, denoted 'ijk', unitless
- the coordinates with physical units in scanner or atlas space
- the coordinate system of the digitization device

To keep track we use the name of the second dimension to store an identifier for CRS.

In [2]:
elements = cedalion.io.read_snirf("../../data/BIDS-NIRS-Tapping/sub-01/nirs/sub-01_task-tapping_nirs.snirf")
geo3d_meas = elements[0].geo3d
geo3d_meas = geo3d_meas.points.rename({"NASION" : "Nz"})
geo3d_meas = geo3d_meas.rename({"pos" : "digitized"})
display(geo3d_meas)

Magnitude,[[-0.041613204679326624 0.026799775287857947 0.1299043936308115] [-0.06476686499872276 0.05814256998996063 0.0908425773727145] [-0.07120554551675068 -0.012874272652217859 0.10787860947691345] [-0.0859043654400404 0.018971698468891116 0.06509762433137256] [0.03694171596700852 0.02748380530252158 0.13022129709104263] [0.06065133742692848 0.05882414589197514 0.09117717995727878] [0.06712771392323756 -0.012199231886346213 0.1085725493643022] [0.08188685574250908 0.020427932162352107 0.06571325110115192] [-0.037619588707178915 0.06322851630256272 0.11572802770110814] [-0.04134445059646741 -0.011779611291995052 0.13495002938154654] [-0.07242424650162711 0.02347293206381116 0.10322218957482163] [-0.07912592748234686 0.05140929117919257 0.057370046083468226] [0.03352717285472944 0.06359968341212022 0.11583881331702946] [0.03686639505686032 -0.011397164907962862 0.13536724076864515] [0.06791592703520163 0.02468254467119271 0.10366605207860985] [0.075310088095807 0.05226884499005337 0.05787698428594235] [-0.03773895423262196 0.034082658086024245 0.1294919790818403] [-0.061454307897075164 0.06443800208211416 0.09061004226260877] [-0.07282878975853647 -0.00527870527992114 0.10743054838539287] [-0.08439610638498087 0.02706123378098264 0.06559510739262155] [0.040013338219712126 0.020439745814301982 0.13063767506528579] [0.06428020193514211 0.05162125732852231 0.09133632943784001] [0.06521393141744246 -0.019260368037897515 0.10880928230870081] [0.08272091030272573 0.012990608473329186 0.06658402323335233] [-0.0824899918305801 3.5272652784690273e-09 -8.985265795291575e-10] [6.534060185275914e-12 0.11404663614484922 -8.956669156345853e-09] [0.08248999697928468 3.893090638057428e-09 4.766247813092761e-10] [-0.04018770669918394 0.044642295725887106 0.12357659157001165] [-0.04174110787598461 0.007685839199884737 0.13437743644514044] [-0.05885642692737942 0.026136335712672674 0.11745327806321545] [0.03851939726517181 0.03078283979366837 0.1281798987708399] [-0.052808259274512416 0.06188780045911764 0.10403189889709587] [-0.06922421433143165 0.04108974212744533 0.0972095427514579] [-0.07351067792317667 0.05556043944468993 0.07592438053279707] [0.031418413207589764 0.05609701242391968 0.12112072355182572] [-0.05798909959739463 -0.013176608236818512 0.12293491786782985] [-0.07419549041066105 0.00549319711704184 0.10669158774707252] [0.023828589288117673 0.003996293896988853 0.14081960191362605] [-0.08184195746940305 0.022098763709400317 0.08422485645365857] [-0.08309090979477994 0.03520830182778492 0.0610323268526085] [0.026425335781627597 0.043375220063173334 0.12939415639453739] [0.035781770725957916 0.04568049698512951 0.12354633505671649] [0.037556277679295876 0.008001190852073497 0.134555406719549] [0.054080633753515184 0.026705026363902817 0.11818464121484251] [0.06801076312164317 0.02284818084680601 0.10445872040876931] [0.0481712114118228 0.062034828924799924 0.10518292837437133] [0.06461419274656584 0.042329086589563227 0.0977770136200937] [0.06967032405092426 0.05604290375898291 0.07590918892077606] [0.07526945345643418 0.037578919664336574 0.08007431341875021] [0.05288202968424921 -0.012139614237985343 0.12430811820847815] [0.06938013615060833 0.0069016618704050535 0.10806432085401242] [0.07932239267332403 -0.00044800545870984573 0.08903670686337778] [0.07756024143390347 0.022515587935674892 0.08578432016555051] [0.07912303758068968 0.036496511669642975 0.06257633565753129] [0.08225821050778642 0.01751423306824721 0.06619028345490524]]
Units,meter


## Read segmented MRI scans

The image cubes are returned as a stacked xr.DataArray. 

In [3]:
DATADIR = "/home/eike/Projekte/ibslab/30_dev/AtlasViewerPy/demo_data"

In [4]:
masks, t_ijk2ras = cedalion.io.read_segmentation_masks(DATADIR+"/anatomy_data")
masks

<xarray.DataArray (segmentation_type: 5, i: 197, j: 233, k: 189)>
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Coordinates:
  * segmentation_type  (segmentation_type) <U5 'csf' 'gm' 'scalp' 'skull' 'wm'
Dimensions without coordinates: i, j, k

Additionaly, a transformation matrix is returned ton convert from voxel space (ijk) to scanner space as it is defined in the niftii files. Since the segmentation masks were derived from a MRI scan, nibabel denotes the coordinate system with the affine code `'aligned'`.

The transformation matrices are also xr.DataArrays that contain both CRS names as dimension names. When applying this transformation to coordinates in voxel space (`'ijk'`) the matrix multiplication will contract the `'ijk'` dimension and the coordinates will have their coordinate dimension named `'aligned'`. The units of the transformation matrix will take care of necessary unit conversions. Here dimensionless in voxel space to millimeter in scanner space.

In [5]:
t_ijk2ras # transform from voxel space (ijk) to scanner space (x=Right y=Anterior z=Superior)

Magnitude,[[1.0 0.0 0.0 -98.0] [0.0 1.0 0.0 -134.0] [0.0 0.0 1.0 -72.0] [0.0 0.0 0.0 1.0]]
Units,millimeter


## Derive surfaces from segmentations

In [6]:
pial_surface = cedalion.geometry.segmentation.surface_from_segmentation(masks, ["wm", "gm"])
pial_surface = pial_surface.apply_transform(t_ijk2ras)

scalp_surface = cedalion.geometry.segmentation.surface_from_segmentation(
    masks, 
    masks.segmentation_type.values, # select all
    fill_holes_in_mask=True)
scalp_surface = scalp_surface.apply_transform(t_ijk2ras)
display(scalp_surface)

TrimeshSurface(mesh=<trimesh.Trimesh(vertices.shape=(167864, 3), faces.shape=(334620, 3))>, crs='aligned', units=<Unit('millimeter')>)

## Load landmarks of the loaded scan.

These were handpicked are define a reference to which the otopde positions should be registered. 

In [7]:
geo3d_volume = cedalion.io.read_mrk_json(DATADIR+"/anatomy_data/landmarks.mrk.json", crs="aligned")
geo3d_volume

Magnitude,[[0.8874586820602417 87.58612060546875 -45.87991714477539] [0.4660395681858063 -120.75855255126953 -48.621673583984375] [-86.02318572998047 -21.7824764251709 -51.1181755065918] [87.26730346679688 -20.213722229003906 -51.89456558227539]]
Units,millimeter


## Simple registration algorithm
Find an affine transformation that translates and rotates the optode coordinates to match the landmarks.
Scaling is allowed only to transform units.

In [8]:
trafo = cedalion.geometry.registration.register_trans_rot(geo3d_volume, geo3d_meas)
display(trafo)
cedalion.plots.plot3d(None, scalp_surface.mesh, geo3d_meas.points.apply_transform(trafo), None) 

Magnitude,[[999.9795742361432 -4.980361091603256 -4.0058849083440755 0.8998610073240974] [4.766479451359034 998.6493708067931 -51.73697769329185 -22.76755935444356] [-4.258143273943055 51.71682695790269 998.6527114193727 -51.59693064415681] [0.0 0.0 0.0 1.0]]
Units,millimeter/meter


Widget(value='<iframe src="http://localhost:35329/index.html?ui=P_0x7f907cb0a3e0_0&reconnect=auto" class="pyvi…

## Snap points to closest vertex on the scalp surface

In [9]:
snapped = scalp_surface.snap(geo3d_meas.points.apply_transform(trafo))
cedalion.plots.plot3d(None, scalp_surface.mesh, snapped, None) 

Widget(value='<iframe src="http://localhost:35329/index.html?ui=P_0x7f907bf87dc0_1&reconnect=auto" class="pyvi…

## Compare common landmarks in both point sets

In [10]:
common = snapped.points.common_labels(geo3d_volume)
display(geo3d_volume.sel(label=common))
display(snapped.sel(label=common))

Magnitude,[[87.26730346679688 -20.213722229003906 -51.89456558227539] [-86.02318572998047 -21.7824764251709 -51.1181755065918] [0.8874586820602417 87.58612060546875 -45.87991714477539]]
Units,millimeter


Magnitude,[[86.10000610351562 -23.0 -52.0] [-84.10000038146973 -25.0 -51.0] [0.0 87.10000610351562 -46.0]]
Units,millimeter


## Transform registered optode locations back to voxel space

In [11]:
t_ras2ijk = cedalion.xrutils.pinv(t_ijk2ras)
snapped.points.apply_transform(t_ras2ijk).round()

Magnitude,[[52.0 134.0 157.0] [32.0 165.0 116.0] [21.0 92.0 130.0] [13.0 126.0 87.0] [141.0 134.0 159.0] [163.0 167.0 116.0] [173.0 90.0 133.0] [183.0 129.0 87.0] [58.0 172.0 143.0] [51.0 92.0 163.0] [23.0 129.0 127.0] [19.0 159.0 80.0] [135.0 172.0 145.0] [142.0 93.0 165.0] [171.0 133.0 128.0] [176.0 161.0 81.0] [56.0 141.0 157.0] [35.0 172.0 115.0] [20.0 100.0 131.0] [14.0 134.0 88.0] [144.0 128.0 159.0] [167.0 160.0 116.0] [172.0 84.0 132.0] [184.0 121.0 87.0] [14.0 109.0 21.0] [98.0 221.0 26.0] [184.0 111.0 20.0] [54.0 152.0 151.0] [52.0 114.0 163.0] [35.0 131.0 143.0] [142.0 138.0 157.0] [42.0 168.0 130.0] [26.0 147.0 121.0] [24.0 163.0 99.0] [132.0 165.0 151.0] [34.0 89.0 148.0] [20.0 112.0 129.0] [127.0 111.0 172.0] [16.0 129.0 107.0] [16.0 143.0 84.0] [128.0 151.0 160.0] [138.0 154.0 153.0] [142.0 115.0 164.0] [158.0 134.0 145.0] [172.0 130.0 128.0] [150.0 172.0 132.0] [168.0 151.0 122.0] [171.0 165.0 99.0] [177.0 146.0 103.0] [159.0 92.0 152.0] [174.0 115.0 132.0] [183.0 107.0 109.0] [180.0 130.0 107.0] [180.0 145.0 84.0] [183.0 126.0 87.0]]
Units,dimensionless


## ICP registration [WIP]

In [12]:
losses, trafos = cedalion.geometry.registration.register_icp(scalp_surface, geo3d_volume, elements[0].geo3d)

p.plot(losses)

AttributeError: 'numpy.ndarray' object has no attribute 'points'

In [ ]:
reg2 = elements[0].geo3d.points.apply_transform(trafos[-1])
cedalion.plots.plot3d(None, scalp_surface, reg2, None)
display(trafos[-1])

In [ ]:
simple_scalp = surface.as_trimesh().simplify_quadric_decimation(60e3)
simple_brain = pial_surface.simplify_quadric_decimation(60e3)

In [ ]:
brain_mask = masks.sel(segmentation_type=["gm", "wm"]).sum("segmentation_type")

In [ ]:
cell_coords = cedalion.imagereco.geometry.cell_coordinates(brain_mask, t_vox2ras).stack({"cell" : ["i","j","k"]})

In [ ]:
from scipy.spatial import KDTree
t = KDTree(simple_brain.vertices)

In [ ]:
cell_indices = np.flatnonzero(brain_mask.values)
dists, vertex_indices = t.query(cell_coords[:,indices].values.T, workers=-1)

In [ ]:
cell_indices

In [ ]:
import scipy.sparse
scipy.sparse.coo_matrix?

In [ ]:
ncells = np.prod(brain_mask.shape)
nvertices = len(simple_scalp.vertices)
Mcoo = scipy.sparse.coo_array((np.ones(len(cell_indices)), (vertex_indices, cell_indices)), shape=(nvertices, ncells)) 
Mcsr = scipy.sparse.csr_array((np.ones(len(cell_indices)), (vertex_indices, cell_indices)), shape=(nvertices, ncells)) 

In [ ]:
test = np.arange(ncells)

In [ ]:
%timeit (Mcoo @ test)

In [ ]:
%timeit (Mcsr @ test)

In [ ]:
t_ras2vox = np.linalg.pinv(t_vox2ras).round(12)

In [ ]:
reg2.pint.to("mm").points.apply_transform(t_ras2vox).max("label")

In [ ]:
[str(i) for i in geo3d_volume.label.values]

In [ ]:
geo3d_volume.pint.to("mm").pint.dequantify().values

In [ ]:
#trimesh.smoothing.filter_taubin(pial_surface, lamb=0.5).show()
#pial_surface_low = pial_surface.simplify_quadric_decimation(60e3)

#display(pial_surface)
#display(pial_surface_low)
#trimesh.smoothing.filter_taubin(pial_surface.mesh, lamb=0.5)

In [ ]:
# calculate median tri size
#tri = pial_surface_low.vertices[pial_surface_low.faces]
#a = np.linalg.norm(tri[:,1,:] - tri[:,0,:], axis=1)
#b = np.linalg.norm(tri[:,2,:] - tri[:,0,:], axis=1)
#A = a*b/2
#np.median(A), np.std(A)